In [4]:
%matplotlib inline
import math as math
import pandas as pd
import numpy as np
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import ipywidgets

In [9]:
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense

Using TensorFlow backend.
c:\users\benson\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\benson\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\benson\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [40]:
folderpath = "*"
folderpath = "C:\\Users\\Benson\\Desktop"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st = pd.read_csv(file)

ft_columns = ['timestamp',
 'agent_index_interval_start',
 'agent_index_interval_end',
 'ego_translation_x',
 'ego_translation_y',
 'ego_translation_z',
 'traffic_lights_start',
 'traffic_lights_end',
 'ego_rotation_xx',
 'ego_rotation_xy',
 'ego_rotation_xz',
 'ego_rotation_yx',
 'ego_rotation_yy',
 'ego_rotation_yz',
 'ego_rotation_zx',
 'ego_rotation_zy',
 'ego_rotation_zz',
 'frame_index',
 'scene_index',
 'Mean Distance from Agents']
try:
    assert list(ft.columns) == ft_columns
except:
    execute_frametable_column_preparation()

In [59]:
class Frames_Column_Creator (dict):
    def __getitem__(self, attr):
        if attr == 'scene_index':
            return self.create_frames_scene_index
        if attr == 'frame_index':
            return self.create_frames_frame_index
        if attr == 'Mean Distance from Agents':
            return self.create_frames_mean_distance_agents
    def create_frames_frame_index(self, st = st):
        frame_indexer = []
        for i in range(len(st)):
            row = st.iloc[i]
            frame_indices = list(range(row['frame_index_interval_start'], row['frame_index_interval_end']))
            frame_indexer = frame_indexer + frame_indices
        ft['frame_index'] = frame_indexer
        return frame_indexer.values
    def create_frames_scene_index(self, st = st):
        scene_indexer = []
        for i in range(len(st)):
            row = st.iloc[i]
            amount = row['frame_index_interval_end'] - row['frame_index_interval_start']
            sublst = [i for _ in range(amount)]
            scene_indexer = scene_indexer + sublst
        ft['scene_index'] = scene_indexer
        return scene_indexer.values
    def create_frames_mean_distance_agents(self, at = at):
        at_temp = at[['centroid_x','centroid_y','frame_index']]
        ft_temp = ft[['ego_translation_x', 'ego_translation_y', 'frame_index']]
        at_temp = at_temp.merge(ft_temp, on="frame_index")
        dx = (at_temp['centroid_x'] - at_temp['ego_translation_x'])**2
        dy = (at_temp['centroid_y'] - at_temp['ego_translation_y'])**2
        dist = (dx + dy)**(1/2)
        at_temp['dist'] = dist
        meandist = at_temp.groupby('frame_index',sort=False).mean()['dist']
        ft['Mean Distance from Agents'] = meandist.values
        return meandist.values
class Agents_Column_Creator (dict):
    def __getitem__(self, attr):
        if attr == 'size':
            return self.create_agents_size
        if attr == 'frame_index':
            return self.create_agents_frame_index
    def create_agents_size(self, at = at):
        size = (at['extent_x'] * at['extent_y'])
        at['size'] = size.values
        return size.values
    def create_agents_frame_index(self, at = at):
        frame_indexer = []
        for i in range(len(ft)):
            row = ft.iloc[i]
            amount = row['agent_index_interval_end'] - row['agent_index_interval_start']
            sublst = [i for _ in range(int(amount))]
            frame_indexer = frame_indexer + sublst
        at['frame_index'] = frame_indexer
        return frame_indexer.values   

In [60]:
frames_column_creator = Frames_Column_Creator()
agents_column_creator = Agents_Column_Creator()

In [61]:
ft_necessary_columns = ['scene_index', 'frame_index', 'Mean Distance from Agents']
for colname in ft_necessary_columns:
    if colname in ft.columns:
        pass
    else:
        print("creating {0}".format(colname))
        create_column_func = frames_column_creator[colname]
        create_column_func()
at_necessary_columns = ['frame_index', 'size']
for colname in at_necessary_columns:
    if colname in at.columns:
        pass
    else:
        print("creating {0}".format(colname))
        create_column_func = agents_column_creator[colname]
        create_column_func()

### Preparing Frame instances
* Group together the agents with their frame in agent table, using groupby unsorted
* For each group of agents, aggregate the average values of the columns: size, velocity_x, velocity_y, yaw
* This should collapse the agent table into a table that's n rows long and 5 columns wide, with n = height of frame table
* Append this table to the frame table by merging with frame_index
* Call the new merged table, ft2. This new table is now ready to be processed into predictions and targets

In [63]:
class Table_Aggregation:
    def __init__(self):
        pass
    def aggregator(self, table: pd.DataFrame = None) -> np.array:
        """ Collapses a table of a certain frame_index in a source table into an array
        Args:
            table (pd.DataFrame): the subset table of the source table with a certain frame_index
        Returns:
            np.array: the horizontal aggregation of the columns in the subsettted table
        """
        return None

In [64]:
class Agent_Aggregation (Table_Aggregation):
    def __init__(self, table: pd.DataFrame = at, columns: [str] = []):
        super(Agent_Aggregation, self).__init__()
        self.table = table # agent_table
        self.columns = columns
        subset_table = table[columns].copy() #[table.frame_index == frame_index][columns].copy()
        self.subset_table = subset_table
    def aggregator(self, table: pd.DataFrame = None) -> pd.DataFrame:
        if table == None:
            table = self.subset_table 
        table['temp_index'] = table['frame_index']
        avgs = table.groupby('temp_index', sort=False).mean()
        return avgs

In [65]:
def create_ft2(ft=ft):
    nn1 = Agent_Aggregation(table = at,
                            columns = [
                                "frame_index",
                                "size",
                                "velocity_x",
                                "velocity_y",
                                "yaw"
                            ])
    nn1_table = nn1.aggregator()
    ft2 = ft.merge(nn1_table)
    return ft2

In [66]:
ft2 = create_ft2(ft)

ft2 = ft2.drop([
    'timestamp', 
    'agent_index_interval_start', 
    'agent_index_interval_end',
    'traffic_lights_start', 
    'traffic_lights_end',
    'frame_index'], axis = 1)

In [69]:
FT2_PRE_INPUT_COLUMN_ORDER = ['ego_translation_x',
 'ego_translation_y',
 'ego_translation_z',
 'ego_rotation_xx',
 'ego_rotation_xy',
 'ego_rotation_xz',
 'ego_rotation_yx',
 'ego_rotation_yy',
 'ego_rotation_yz',
 'ego_rotation_zx',
 'ego_rotation_zy',
 'ego_rotation_zz',
 'scene_index',
 'Mean Distance from Agents',
 'size',
 'velocity_x',
 'velocity_y',
 'yaw']
# ft2's columns MUST BE in this order before we convert each row into an 1x24 input for the neural network.
if (FT2_PRE_INPUT_COLUMN_ORDER) == (list(ft2.columns)):
    # ensure that order is the same
    pass
else:
    # colnames all accounted for, but out of order
    ft2 = ft2[FT2_PRE_INPUT_COLUMN_ORDER]

### Preparing Frame instances (Input preparation)
* For each scene in ft2:
* pair up each frame-row with previous frame (so the first frame is usually discarded)
* create a Frame instance object, with the prev frame-row, and the discrete angle of the current frame-row
* put into dataset array frame_instances, which we will later split apart and split into inputs and labels

In [70]:
class Frame_Instance:
    def __init__(self, frame_0_array: np.array, frame_1_transl: np.array):
        self.frame_0_array = frame_0_array
        self.frame_1_translation = frame_1_transl

In [71]:
def angle_discrete(dx, dy):
    half_angle = np.arctan2(dy, dx)
    if half_angle < 0:
        # print("too small,", half_angle)
        theta = (math.pi + half_angle) + math.pi
    else:
        theta = half_angle
    assert theta >= 0
    assert theta < 2*math.pi
    octer = theta // (math.pi/4)
    return int(octer)

In [72]:
def frame_encoder(frame_0_row, frame_1_row):
    input_ = np.array(frame_0_row)
    x0, y0 = frame_0_row[['ego_translation_x', 'ego_translation_y']].values
    x1, y1 = frame_1_row[['ego_translation_x', 'ego_translation_y']].values
    dx = x1 - x0
    dy = y1 - y0
    one_hot_encoder = np.zeros(8)
    octer = angle_discrete(dx, dy)
    one_hot_encoder[octer] = 1
    return input_, one_hot_encoder

In [74]:
unique_scenes_indices = pd.unique(ft2['scene_index'])
frame_instances = np.array([])

for s_i in range(len(unique_scenes_indices)):
    scene_index = unique_scenes_indices[s_i]
    if s_i % 10 == 0:
        print('{0} of 100 scenes done'.format(s_i))
    ft2_scene = ft2[ft2.scene_index == scene_index]
    for i in range(1, len(ft2_scene)):
        frame_1_row = ft2_scene.iloc[i]
        frame_0_row = ft2_scene.iloc[i - 1].drop("scene_index")
        input_, label = frame_encoder(frame_0_row, frame_1_row)
        frame_instance = Frame_Instance(input_, label)
        frame_instances = np.append(frame_instances, frame_instance)

0 of 100 scenes done
10 of 100 scenes done
20 of 100 scenes done
30 of 100 scenes done
40 of 100 scenes done
50 of 100 scenes done
60 of 100 scenes done
70 of 100 scenes done
80 of 100 scenes done
90 of 100 scenes done


In [89]:
ft2

,ego_translation_x,ego_translation_y,ego_translation_z,ego_rotation_xx,ego_rotation_xy,ego_rotation_xz,ego_rotation_yx,ego_rotation_yy,ego_rotation_yz,ego_rotation_zx,ego_rotation_zy,ego_rotation_zz,scene_index,Mean Distance from Agents,size,velocity_x,velocity_y,yaw
0,687.888733,-2161.516602,290.123901,-0.548663,0.829618,-0.103456,-0.831995,-0.553972,-0.029972,-0.082177,0.069631,0.994182,7827,29.661129,3.164270,-0.000697,0.001090,0.514642
1,687.285095,-2162.442627,290.042358,-0.548607,0.829808,-0.102218,-0.832065,-0.553844,-0.030399,-0.081839,0.068375,0.994297,7827,29.767024,2.391519,0.206666,0.275784,0.494815
2,686.678772,-2163.348145,289.963165,-0.548695,0.830226,-0.098282,-0.831977,-0.553805,-0.033390,-0.082151,0.063448,0.994598,7827,29.749838,2.400336,0.165006,0.284153,0.560013
3,686.072205,-2164.247314,289.893311,-0.549262,0.830413,-0.093408,-0.831583,-0.554178,-0.036822,-0.082342,0.057452,0.994947,7827,29.634852,1.986993,0.179429,0.235891,0.505815
4,685.482300,-2165.144531,289.814209,-0.548979,0.830953,-0.090218,-0.831735,-0.553773,-0.039398,-0.082698,0.053408,0.995143,7827,28.864490,1.741472,0.157543,0.280815,0.527288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,-543.666443,903.855469,271.994934,0.681983,0.730755,-0.029941,-0.731367,0.681338,-0.029678,-0.001287,0.042138,0.999111,15427,41.851427,5.012744,-0.377695,-0.639061,-0.169691
24833,-543.666443,903.855469,271.994934,0.681983,0.730755,-0.029941,-0.731367,0.681338,-0.029678,-0.001287,0.042138,0.999111,15427,43.580053,5.145728,-0.788369,-0.208533,-0.293852
24834,-543.666443,903.855469,271.994934,0.681983,0.730755,-0.029941,-0.731367,0.681338,-0.029678,-0.001287,0.042138,0.999111,15427,42.984665,5.145540,-0.947997,-0.099681,-0.104517
24835,-543.666443,903.855469,271.994934,0.681983,0.730755,-0.029941,-0.731367,0.681338,-0.029678,-0.001287,0.042138,0.999111,15427,44.383615,4.996338,-0.852352,-0.148939,-0.053815


In [76]:
assert len(frame_instances) == 24737

### Designating Training and Testing Data
* Shuffle the list of samples
* Split the first 80% of samples as training dataset, and the last 20% as testing dataset
* For each dataset, split them into the x list (input), and the y list (labels)

In [83]:
TRAIN_PROPORTION = 0.8
# TRAIN_SIZE_INDEXER = int(len(frame_instances) * TRAIN_PROPORTION)

def Designate_training_testing_datasets(list_samples: np.ndarray, 
                                        train_proportion: float = TRAIN_PROPORTION,
                                        shuffling: bool = True) -> tuple:
    assert train_proportion > 0 and train_proportion < 1
    np_frame_instances = list_samples
    if shuffling:
        np.random.shuffle(np_frame_instances)
    
    TRAIN_SIZE_INDEXER = int(len(frame_instances) * train_proportion)

    Train_Dataset = np_frame_instances[:TRAIN_SIZE_INDEXER]
    Test_Dataset = np_frame_instances[TRAIN_SIZE_INDEXER:]
    
    return (Train_Dataset, Test_Dataset)

In [84]:
def Split_dataset_inputs_labels(dataset: np.ndarray) -> np.ndarray:
    x = []
    y = []
    for frame_instance in Train_Dataset:
        xi = frame_instance.frame_0_array
        x.append(xi)
        yi = frame_instance.frame_1_translation
        y.append(yi)
    return np.array(x), np.array(y)

In [85]:
Train_Dataset, Test_Dataset = Designate_training_testing_datasets(frame_instances)

In [86]:
train_x, train_y = Split_dataset_inputs_labels(Train_Dataset)
test_x, test_y = Split_dataset_inputs_labels(Test_Dataset)

np_frame_instances = np.array(frame_instances)
np.random.shuffle(np_frame_instances)
TRAIN_PROPORTION = 0.8
TRAIN_SIZE_INDEXER = int(len(frame_instances) * TRAIN_PROPORTION)
Train_Dataset = np_frame_instances[:TRAIN_SIZE_INDEXER]
Test_Dataset = np_frame_instances[TRAIN_SIZE_INDEXER:]

train_x = []
train_y = []
for frame_instance in Train_Dataset:
    train_xi = frame_instance.frame_0_array
    train_x.append(train_xi)
    train_yi = frame_instance.frame_1_translation
    train_y.append(train_yi)
    
test_x = []
test_y = []
for frame_instance in Test_Dataset:
    test_xi = frame_instance.frame_0_array
    test_x.append(test_xi)
    test_yi = frame_instance.frame_1_translation
    test_y.append(test_yi)
    
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

### Neural Network
* You now have train_x, train_y, and test_x, test_y.

In [87]:
model = tf.keras.Sequential()
model.add(Dense(16, input_shape=(17,)))
model.add(Dense(14))
model.add(Dense(12))
model.add(Dense(10))
model.add(tf.keras.layers.Dense(8, activation=tf.nn.softmax))
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy')

In [88]:
model.fit(train_x, train_y, epochs = 10)
test_loss = model.evaluate((test_x), (test_y))

Epoch 1/10
19789/19789 [==============================] - 1s 45us/sample - loss: 15.5600
Epoch 2/10
19789/19789 [==============================] - 1s 40us/sample - loss: 2.1918
Epoch 3/10
19789/19789 [==============================] - 1s 40us/sample - loss: 1.9623
Epoch 4/10
19789/19789 [==============================] - 1s 40us/sample - loss: 1.5248
Epoch 5/10
19789/19789 [==============================] - 1s 40us/sample - loss: 1.3841
Epoch 6/10
19789/19789 [==============================] - 1s 48us/sample - loss: 1.2889
Epoch 7/10
19789/19789 [==============================] - 1s 41us/sample - loss: 1.2357
Epoch 8/10
19789/19789 [==============================] - 1s 42us/sample - loss: 1.1832
Epoch 9/10
19789/19789 [==============================] - 1s 43us/sample - loss: 1.1669
Epoch 10/10
19789/19789 [==============================] - 0s 24us/sample - loss: 0.7698


* Note that test_x and test_y can be changed with your own data
* To do that, get a frames table of your choice 
* convert it have the columns of ft2, in the proper order, 
* run it thru all the cells of  Preparing Frame instances (Input preparation)
* then in the process "Designating training and testing", immediately skip Designate_training_testing_datasets()
* set Testing Dataset as frame_instances, and run it thru Split_dataset_inputs_labels()